<a href="https://colab.research.google.com/github/td-yoshimura/hivemall-notebooks/blob/master/notebooks/misc/text_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
In this notebook, there are some samples for text tokenizing functions in Hivemall.

In [0]:
%env TD_API_KEY = YOUR TD API KEY
%env DATABASE = YOUR DATABASE NAME

<font color="red"><b>You will be asked to input UserName and Password of Treasure Data. Please fill them.</b></font>

In [0]:
!curl -L https://toolbelt.treasuredata.com/sh/install-ubuntu-trusty-td-agent2.sh | sh
!td account -f
!pip install pandas-td

In [0]:
import os
td_api_key = os.environ['TD_API_KEY']
database = os.environ['DATABASE']
table = 'churn_dataset'

import numpy as np
import pandas as pd

import pandas_td as td
con = td.connect(apikey=td_api_key)

%load_ext pandas_td.ipython
%td_use $database

presto = td.create_engine('presto:{}'.format(database), con=con)
hive = td.create_engine('hive:{}'.format(database), con=con)

# Japanes Text

tokenize_ja and tokenize_ja_neologd's arguments are below,
- 1st argument: text
- 2nd argument: segmentation modes ()normal/search/extended) http://www.atilika.org/
- 3rd argument: stop words
- 4th argument: stop tags https://qiita.com/miz21358/items/75fd28075ed8cfe2c937

In [0]:
%%td_hive
select
  *
from (
  select 1 as index, tokenize_ja_neologd("kuromojiを使った分かち書きのテストです。😀", "normal")
  union all
  select 2 as index, tokenize_ja_neologd("kuromojiを使った分かち書きのテストです。😀", "normal", array("kuromoji"))
  union all  
  select 3 as index, tokenize_ja_neologd("kuromojiを使った分かち書きのテストです。😀", "normal", null, array("名詞-一般"))
  union all
  select 4 as index, tokenize_ja_neologd("kuromojiを使った分かち書きのテストです。😀", "normal", array("kuromoji"), array("名詞-一般"))
) t
order by index

,index,_c1
0,1,"[kuromoji, 使う, 分かち書き, テスト, 笑顔]"
1,2,"[使う, 分かち書き, テスト, 笑顔]"
2,3,"[使う, テスト]"
3,4,"[を, 使う, た, の, テスト, です]"


## The diffrence between `tokenize_ja` and `tokenize_ja_neologd`
`tokenize_ja_neologd` has a newer version's dictionary.

In [0]:
%%td_hive
select 'tokenize_ja' as f, tokenize_ja("中居正広のミになる図書館。") as result
union all
select 'tokenize_ja_neologd' as f, tokenize_ja_neologd("中居正広のミになる図書館。") as result

,f,result
0,tokenize_ja,"[中居, 正広, ミ, 図書館]"
1,tokenize_ja_neologd,[中居正広の身になる図書館]


# English Text

In [0]:
%%td_hive
select
  *
from (
  select 1 as index, tokenize("I am studying Hivemall because it's a lot of fun", true)
  union all
  select 2 as index, tokenize("I am studying Hivemall because it's a lot of fun", false)
  union all
  select 3 as index, tokenize("I am studying Hivemall because it's a lot of fun") -- default value of the second argument is false
) t
order by index

,index,_c1
0,1,"[i, am, studying, hivemall, because, it, s, a,..."
1,2,"[I, am, studying, Hivemall, because, it, s, a,..."
2,3,"[I, am, studying, Hivemall, because, it, s, a,..."


# Chinese Text

In [0]:
%%td_hive
select 
  tokenize_cn("Smartcn为Apache2.0协议的开源中文分词系统，Java语言编写，修改的中科院计算所ICTCLAS分词系统。")

,_c0
0,"[smartcn, 为, apach, 2, 0, 协议, 的, 开源, 中文, 分词, 系..."
